In [1]:
import torch
import json

from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

c:\Users\kauhe\anaconda3\envs\authorship-problem\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# LOAD DATA FROM DISK

x_inputs = torch.load('./torch-cache/x_inputs.pt', weights_only=True)
x_masks = torch.load('./torch-cache/x_masks.pt', weights_only=True)
y_labels = torch.load('./torch-cache/y_labels.pt', weights_only=True)

with open('./torch-cache/authors.json', 'r') as file:
    authors = json.load(file)

In [3]:
# PUT DATA INTO TORCH DATALOADER

print("INPUT + LABEL SHAPES: ", x_inputs.shape, x_masks.shape, y_labels.shape)

dataset = TensorDataset(x_inputs, x_masks, y_labels)
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

batch_size = 16
train_dataloader = DataLoader(
    train_dataset,
    sampler=RandomSampler(train_dataset),
    batch_size=batch_size
)
validation_dataloader = DataLoader(
    val_dataset,
    sampler=SequentialSampler(val_dataset),
    batch_size=batch_size
)

INPUT + LABEL SHAPES:  torch.Size([19043, 128]) torch.Size([19043, 128]) torch.Size([19043])
17,138 training samples
1,905 validation samples


In [4]:
for step, batch in enumerate(train_dataloader):
    b_input_ids = batch[0]
    b_input_mask = batch[1]
    b_labels = batch[2]

    print(b_input_ids[0])
    print(len(b_input_ids[0]))

    break

tensor([  100,  1045,  3305,  2017,  2025,  2026,  2935,  1045,  2572,  5580,
         1997,  2009,  1037, 14161, 18891,  4095,  4613, 25126,  1999,  1037,
        13219,  4540,  2026,  2935,  2017,  2442,  2425,  2149,  2073,  1996,
         2303,  2003,  1998,  2175,  2007,  2149,  2000,  1996,  2332,  1996,
         2303,  2003,  2007,  1996,  2332,  2021,  1996,  2332,  2003,  2025,
         2007,  1996,  2303,  1996,  2332,  2003,  1037,  2518,  1037,  2518,
         2026,  2935,  3288,  2033,  2000,  2032,  5342,  4419,  1998,  2035,
         2044,  2178,  2282,  1999,  1996,  3317,  4607,  2332,  3230,  1045,
         2031,  2741,  2000,  6148,  2032,  1998,  2000,  2424,  1996,  2303,
         2129,  4795,  2003,  2009,  2008,  2023,  2158,  3632,  6065,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0])

In [5]:
# USE PRE-TRAINED BERT MODEL

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels=len(authors),
    output_attentions=False,
    output_hidden_states=False,
)

model = model.to(torch.device("cuda" if torch.cuda.is_available() else "cpu"))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# SET UP HYPERPARAMETERS

optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)
epochs = 2 # shoudl be 2-4
total_steps = len(train_dataloader) * epochs
scheduler = get_linear_schedule_with_warmup(optimizer,  num_warmup_steps = 0, num_training_steps = total_steps)

c:\Users\kauhe\anaconda3\envs\authorship-problem\Lib\site-packages\transformers\optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [7]:
# HELPER FUNCTIONS

import numpy as np
import time
import datetime
import random

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [8]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

all_labels = np.concatenate([batch[2].numpy() for batch in train_dataloader])
class_weights = compute_class_weight(
    class_weight='balanced',  # Option to balance automatically
    classes=np.unique(all_labels),
    y=all_labels
)
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda' if torch.cuda.is_available() else 'cpu')
print("Class weights:", class_weights)

Class weights: tensor([0.3065, 0.7804, 0.8385, 3.2440, 4.5556, 0.8283, 1.2060, 2.3081, 3.7559],
       device='cuda:0')


In [9]:
# MAIN TRAINING LOOP
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

seed_val = 42
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []
total_t0 = time.time()

for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dataloader):
        if step % 40 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        model.zero_grad()        

        outputs = model(b_input_ids, 
                             token_type_ids=None, 
                             attention_mask=b_input_mask, 
                             labels=b_labels)
        # loss = outputs.loss
        logits = outputs.logits
        loss_fn = torch.nn.CrossEntropyLoss(weight=class_weights)
        loss = loss_fn(logits, b_labels)

        total_train_loss += loss.item()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)   
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    print("")
    print("Running Validation...")

    t0 = time.time()
    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dataloader:
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        with torch.no_grad():        
            outputs = model(b_input_ids, 
                                   token_type_ids=None, 
                                   attention_mask=b_input_mask,
                                   labels=b_labels)
            loss = outputs.loss
            logits = outputs.logits
            
        total_eval_loss += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        total_eval_accuracy += flat_accuracy(logits, label_ids)
        
    avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
    print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

    avg_val_loss = total_eval_loss / len(validation_dataloader)
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Accur.': avg_val_accuracy,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

cuda

======== Epoch 1 / 2 ========
Training...
  Batch    40  of  1,072.    Elapsed: 0:00:02.
  Batch    80  of  1,072.    Elapsed: 0:00:04.
  Batch   120  of  1,072.    Elapsed: 0:00:06.
  Batch   160  of  1,072.    Elapsed: 0:00:09.
  Batch   200  of  1,072.    Elapsed: 0:00:11.
  Batch   240  of  1,072.    Elapsed: 0:00:13.
  Batch   280  of  1,072.    Elapsed: 0:00:15.
  Batch   320  of  1,072.    Elapsed: 0:00:17.
  Batch   360  of  1,072.    Elapsed: 0:00:19.
  Batch   400  of  1,072.    Elapsed: 0:00:21.
  Batch   440  of  1,072.    Elapsed: 0:00:23.
  Batch   480  of  1,072.    Elapsed: 0:00:25.
  Batch   520  of  1,072.    Elapsed: 0:00:27.
  Batch   560  of  1,072.    Elapsed: 0:00:29.
  Batch   600  of  1,072.    Elapsed: 0:00:32.
  Batch   640  of  1,072.    Elapsed: 0:00:34.
  Batch   680  of  1,072.    Elapsed: 0:00:36.
  Batch   720  of  1,072.    Elapsed: 0:00:38.
  Batch   760  of  1,072.    Elapsed: 0:00:40.
  Batch   800  of  1,072.    Elapsed: 0:00:42.
  Batch   84

In [ ]:
# SAVE FINE-TUNED MODEL TO DISK

torch.save(model, './torch-cache/model_e2_b_cw1_s128.ckpt')
# model
# model_e4_b_cw1_s128 -> 0.85
# model_e3_b_cw1_s128 -> 0.81
# model_e2_b_cw1_s128 -> 0.80